In [2]:
import numpy as np
import pandas as pd
import nltk

In [13]:
df = pd.read_csv('fake_news.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [14]:
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [21]:
##Applying data cleaning
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [99]:
def cleanDataset(dataset):
    stemmer = PorterStemmer()
    corpus = []
    for sen in dataset:
        temp_sen = re.sub('[^a-zA-Z]', ' ', sen)
        temp_sen = temp_sen.lower()
        words = temp_sen.split()
        
        words = [stemmer.stem(word) for word in words if not word in stopwords.words('english')]
        words = ' '.join(str(word) for word in words)
        corpus.append(words)
    return corpus

In [33]:
# Create a corpus of titles
corpus =[]

dataset = df['title']
stemmer = PorterStemmer()
for sen in dataset:
    temp_sen = re.sub('[^a-zA-Z]', ' ', sen)
    temp_sen = temp_sen.lower()
    words = temp_sen.split()
    
    words = [stemmer.stem(word) for word in words if not word in stopwords.words('english')]
    words = ' '.join(str(word) for word in words)
    corpus.append(words)

['house', 'dem', 'aide', 'we', 'didn', 't', 'even', 'see', 'comey', 's', 'letter', 'until', 'jason', 'chaffetz', 'tweeted', 'it']
['hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet']
hous dem aid even see comey letter jason chaffetz tweet
['flynn', 'hillary', 'clinton', 'big', 'woman', 'on', 'campus', 'breitbart']
['flynn', 'hillari', 'clinton', 'big', 'woman', 'campu', 'breitbart']
flynn hillari clinton big woman campu breitbart
['why', 'the', 'truth', 'might', 'get', 'you', 'fired']
['truth', 'might', 'get', 'fire']
truth might get fire
['civilians', 'killed', 'in', 'single', 'us', 'airstrike', 'have', 'been', 'identified']
['civilian', 'kill', 'singl', 'us', 'airstrik', 'identifi']
civilian kill singl us airstrik identifi
['iranian', 'woman', 'jailed', 'for', 'fictional', 'unpublished', 'story', 'about', 'woman', 'stoned', 'to', 'death', 'for', 'adultery']
['iranian', 'woman', 'jail', 'fiction', 'unpublish', 'stori', 'woman', 'stone', 'death', 'adul

In [34]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [37]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot

In [38]:
voc_size = 10000

In [45]:
sentences = []
sentences=[one_hot(words,voc_size) for words in corpus]

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [46]:
sen_len = 20
padded_corpus = pad_sequences(sentences, maxlen = sen_len, padding = 'pre')

In [47]:
padded_corpus

array([[   0,    0,    0, ..., 4123, 4603,   89],
       [   0,    0,    0, ..., 9709,  951, 7719],
       [   0,    0,    0, ..., 1215, 5024, 6501],
       ...,
       [   0,    0,    0, ..., 3124, 6016, 5099],
       [   0,    0,    0, ..., 3557, 2966, 3604],
       [   0,    0,    0, ..., 2832, 2193, 9049]])

In [59]:
model = Sequential()

In [60]:
output_dim = 20

In [61]:
model.add(Embedding(voc_size, output_dim, input_length=sen_len))

In [62]:
model.add(LSTM(100))

In [63]:
model.add(Dense(1, 'sigmoid', kernel_initializer='glorot_uniform'))

In [64]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [65]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [66]:
X=np.array(padded_corpus)
Y=np.array(df['label'])

In [67]:
X.shape

(18285, 20)

In [68]:
Y.shape

(18285,)

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [71]:
model.fit(X_train, y_train, epochs = 10, validation_data=(X_test, y_test), batch_size=20)

Epoch 1/10
686/686 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step - accuracy: 0.8417 - loss: 0.3403 - val_accuracy: 0.9191 - val_loss: 0.1926
Epoch 2/10
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.9501 - loss: 0.1236 - val_accuracy: 0.9250 - val_loss: 0.2071
Epoch 3/10
686/686 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9737 - loss: 0.0722 - val_accuracy: 0.9186 - val_loss: 0.2345
Epoch 4/10
686/686 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.9841 - loss: 0.0474 - val_accuracy: 0.9042 - val_loss: 0.2798
Epoch 5/10
686/686 ━━━━━━━━━━━━━━━━━━━━ 23s 20ms/step - accuracy: 0.9886 - loss: 0.0359 - val_accuracy: 0.9090 - val_loss: 0.3363
Epoch 6/10
686/686 ━━━━━━━━━━━━━━━━━━━━ 22s 21ms/step - accuracy: 0.9948 - loss: 0.0198 - val_accuracy: 0.9040 - val_loss: 0.3641
Epoch 7/10
686/686 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.9943 - loss: 0.0198 - val_accuracy: 0.9119 - val_loss: 0.4134
Epoch 8/10
686/686 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.9964 - loss: 0.0142 - v

In [73]:
y_pred = model.predict(X_test) > 0.5

143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [74]:
from sklearn import metrics

In [75]:
metrics.accuracy_score(y_test, y_pred)

0.9039807524059492

In [76]:
test_data = pd.read_csv('fake_news_test.csv')

In [77]:
test_data = test_data.dropna()

In [98]:
len(test_data)

4575

In [100]:
test_corpus = cleanDataset(test_data['title'])

In [101]:
len(test_corpus)

4575

In [102]:
test_sentences = [one_hot(word, voc_size) for word in test_corpus]

In [103]:
test_sentences

[[72, 8784, 1247, 5752, 7301, 6605, 2138, 8754, 3124, 6016, 5099],
 [6855, 115, 3243, 1696, 6889, 3450, 595, 1974, 9936, 9864],
 [6963, 2967, 6338, 173, 4022, 5099, 2664, 3124, 6016, 5099],
 [2933, 6148, 1894, 9007, 5777],
 [8058, 6459, 8320, 4873, 3014, 1927, 6594, 8784, 7719],
 [2636, 8865, 3068, 8568, 8300],
 [7139, 2877, 5989, 4276, 8784, 3540, 2868],
 [2559, 1344, 8110, 6744, 1236],
 [6438, 3815, 2398, 6197, 6823, 1338, 2365],
 [8595, 1426, 3934, 327],
 [5735, 1069, 4622, 7980, 5864, 89, 2689, 8578, 8699, 8784, 5445, 5587, 4530],
 [1449, 9302, 6706, 128, 9021, 4690, 6748, 3124, 6016, 5099],
 [3124,
  8654,
  4883,
  104,
  2029,
  4710,
  9951,
  2193,
  1018,
  5965,
  831,
  3124,
  6016,
  5099],
 [5080, 7916, 3668, 7124, 9864, 7624, 184, 3243, 7719],
 [8784, 8811, 4794, 8211, 9963, 5039, 2953, 5716, 6234],
 [962, 5240, 3904, 6266, 9053, 3681, 1112, 8683, 3124, 6016, 5099],
 [3666, 9302, 3129, 5092, 7833, 5991, 9415, 6248],
 [8137, 7283, 8137, 5716, 6594, 8784, 7032, 8450, 5840

In [104]:
padded_test_corpus = pad_sequences(test_sentences, maxlen = sen_len, padding = 'pre')

In [105]:
x_test_data = np.array(padded_test_corpus)

In [106]:
x_test_data

array([[   0,    0,    0, ..., 3124, 6016, 5099],
       [   0,    0,    0, ..., 1974, 9936, 9864],
       [   0,    0,    0, ..., 3124, 6016, 5099],
       ...,
       [   0,    0,    0, ..., 3124, 6016, 5099],
       [   0,    0,    0, ..., 3124, 6016, 5099],
       [   0,    0,    0, ..., 3124, 6016, 5099]])

In [107]:
y_pred = model.predict(x_test_data) > 0.5

143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [112]:
y_pred

AttributeError: 'numpy.ndarray' object has no attribute 'bool'

In [113]:
test_data.head()

,id,title,author,text,label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",False
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,False
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",False
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,True
6,20806,Pelosi Calls for FBI Investigation to Find Out...,Pam Key,"Sunday on NBC’s “Meet the Press,” House Minori...",False


In [111]:
test_data['label'] = y_pred

TypeError: only length-1 arrays can be converted to Python scalars

In [110]:
test_data.head()

,id,title,author,text,label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",False
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,False
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",False
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,True
6,20806,Pelosi Calls for FBI Investigation to Find Out...,Pam Key,"Sunday on NBC’s “Meet the Press,” House Minori...",False
